In [1]:
import pandas as pd
import numpy as np
import talib as ta

### talib package包含了很多的金融时序数据的算法，在数据分析和量化投资领域使用频率高，然而talib不支持矩阵运算。（talib.abstract method对dataframe的columns有严格要求，不方便使用）
### 以下实现对talib算法进行复现以及支持dataframe运算

In [5]:
data = {}
data['close'] = pd.read_pickle('close.pkl')
data['high'] = pd.read_pickle('high.pkl')
data['low'] = pd.read_pickle('low.pkl')
data['open'] = pd.read_pickle('open.pkl')
data['volume'] = pd.read_pickle('volume.pkl')

### ta.aroonup


In [25]:
def self_aroonup(data,paras):
    period = paras['period']

    value = 100 * ((data['high'].rolling(window=period).apply(np.argmax) + 1) / period)

    return  value

In [26]:
def self_aroondown(data,paras):
    period = paras['period']

    value = 100 * ((data['low'].rolling(window=period).apply(np.argmin) + 1) / period)

    return  value

In [27]:
aroonup = self_aroonup(data,{'period':14})['stock_1']
aroondown = self_aroondown(data,{'period':14})['stock_1']

In [28]:
ta_aroondown,ta_aroonup = ta.AROON(data['high']['stock_1'],data['low']['stock_1'])

## ta.EMA

In [40]:
ta_ema = ta.EMA(data['close']['stock_1'],9)

In [47]:
ema = data['close'].ewm(span=9, adjust=False).mean()['stock_1']

In [50]:
ta_ema - ema

2023-05-01 00:00:00             NaN
2023-05-01 00:05:00             NaN
2023-05-01 00:10:00             NaN
2023-05-01 00:15:00             NaN
2023-05-01 00:20:00             NaN
                           ...     
2023-07-31 23:35:00   -7.275958e-12
2023-07-31 23:40:00   -7.275958e-12
2023-07-31 23:45:00   -7.275958e-12
2023-07-31 23:50:00   -7.275958e-12
2023-07-31 23:55:00   -7.275958e-12
Length: 26496, dtype: float64

## ta.RSI

In [61]:
def self_rsi(data,paras):
    period = paras['period']
    delta = data['close'].diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    gain = up.ewm(com=period - 1, min_periods=period).mean()
    loss = abs(down.ewm(com=period - 1, min_periods=period).mean())
    RS = gain / loss
    value = 100 - (100 / (1 + RS))
    return value

In [64]:
paras = {'period':14}
rsi = self_rsi(data,paras)['stock_1']

In [65]:
ta_rsi = ta.RSI(data['close']['stock_1'],14)

In [66]:
ta_rsi-rsi

2023-05-01 00:00:00             NaN
2023-05-01 00:05:00             NaN
2023-05-01 00:10:00             NaN
2023-05-01 00:15:00             NaN
2023-05-01 00:20:00             NaN
                           ...     
2023-07-31 23:35:00    0.000000e+00
2023-07-31 23:40:00    0.000000e+00
2023-07-31 23:45:00    7.105427e-15
2023-07-31 23:50:00   -7.105427e-15
2023-07-31 23:55:00    7.105427e-15
Length: 26496, dtype: float64